
# Capítulo 6 — Avaliação de Modelos

**Curso:** CECIERJ – IA e ML para Soluções Práticas  
**Objetivo:** comparar diferentes estratégias de avaliação, entender *bias/variance* e inspecionar curvas de aprendizado e calibração de probabilidades.

---
## Passos abordados
1. Conceito de **holdout vs cross-validation**.  
2. Uso de **K-Fold/StratifiedKFold**.  
3. **Learning Curves** para diagnosticar *overfitting/underfitting*.  
4. **Validation Curves** para hiperparâmetros.  
5. **Calibration Curve** para probabilidades.  
6. Interpretação prática dos gráficos.


In [ ]:

import numpy as np, matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

ds = load_breast_cancer()
X, y = ds.data, ds.target
pipe = Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=1000, random_state=42))])


In [ ]:

# Holdout simples
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)
print("Accuracy holdout:", pipe.score(X_test, y_test))

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy")
print("CV média:", scores.mean(), "Desvio:", scores.std())


In [ ]:

from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(
    pipe, X, y, cv=cv, scoring="accuracy", train_sizes=np.linspace(0.1,1.0,5), n_jobs=-1
)
plt.figure()
plt.plot(train_sizes, train_scores.mean(axis=1), 'o-', label="Treino")
plt.plot(train_sizes, val_scores.mean(axis=1), 's-', label="Validação")
plt.xlabel("Tamanho do treino"); plt.ylabel("Acurácia")
plt.title("Learning Curve (LogReg)")
plt.legend(); plt.show()


In [ ]:

from sklearn.model_selection import validation_curve

param_range = [0.01, 0.1, 1, 10, 100]
train_scores, val_scores = validation_curve(
    pipe, X, y, param_name="clf__C", param_range=param_range,
    cv=cv, scoring="accuracy", n_jobs=-1
)
plt.figure()
plt.semilogx(param_range, train_scores.mean(axis=1), 'o-', label="Treino")
plt.semilogx(param_range, val_scores.mean(axis=1), 's-', label="Validação")
plt.xlabel("Parâmetro C (LogReg)"); plt.ylabel("Acurácia")
plt.title("Validation Curve")
plt.legend(); plt.show()


In [ ]:

from sklearn.calibration import CalibrationDisplay

pipe.fit(X_train, y_train)
disp = CalibrationDisplay.from_estimator(pipe, X_test, y_test, n_bins=10)
disp.ax_.set_title("Calibration Curve (LogReg)")
plt.show()



---
## Conclusões
- **Holdout** é rápido, mas pode variar; **Cross-validation** dá média mais estável.  
- **Learning curve** ajuda a diagnosticar se há *underfitting* (treino/validação baixos) ou *overfitting* (gap alto).  
- **Validation curve** mostra sensibilidade a hiperparâmetros.  
- **Calibration curve** avalia se probabilidades estão bem calibradas (importante em risco/saúde).  
- Combine métricas + curvas para decidir ajustes de modelo.
